In [ ]:
import pandas as pd
import glob
import os
import gzip
import eviltransform
from math import radians, cos, sin, asin, sqrt
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import datetime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
indir = r"E:\Traffic_Prediction_Challenge"
os.chdir(indir)
fileList=glob.glob("*gz")

# Parsing Raw Data

In [ ]:
# Study Area Bounding Box
lat1,lon1=eviltransform.wgs2gcj(34.234,108.9415)
lat2,lon2=eviltransform.wgs2gcj(34.241,108.943)

outfile='Trb_chllng.csv'
datas=[]
for filename in fileList[:]:    
    print('working on :', filename)

    df=pd.read_csv(filename,compression='gzip',names=['drID','orID','Time','Lon','Lat'])
    df=df[(df['Lat']>=lat1) & (df['Lat']<=lat2)]
    df=df[(df['Lon']>=lon1) & (df['Lon']<=lon2)]
    datas.append(df)

full_df=pd.concat(datas)
full_df.to_csv(outfile)

# Prepare data for model

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lat1,lon1=eviltransform.gcj2wgs_exact(float(lat1),float(lon1))
    lat2,lon2=eviltransform.gcj2wgs_exact(float(lat2),float(lon2))
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [ ]:
df=pd.read_csv('Trb_chllng.csv')
df['Time']=pd.to_datetime(df['Time'],unit='s')+datetime.timedelta(hours=8)
df=df.sort_values(by='Time')
print('done')

In [ ]:
df['month']=df.apply(lambda x: x.Time.month,axis=1)
df.head()

In [ ]:
grp=df.groupby(by='orID')
print('done')

In [ ]:
speed_df=pd.DataFrame()
speed_df['start_time']=grp.Time.first()
speed_df['end_time']=grp.Time.last()
speed_df['start_lat']=grp.Lat.first()
speed_df['end_lat']=grp.Lat.last()
speed_df['start_lon']=grp.Lon.first()
speed_df['end_lon']=grp.Lon.last()
print('done')

In [ ]:
speed_df['month']=grp.month.first()

In [ ]:
speed_df['time_diff']=speed_df.apply(lambda x: (x['end_time']-x['start_time']).total_seconds()/3600, axis=1)
print('done')

In [ ]:
speed_df['distance']=speed_df.apply(lambda x: haversine(x.start_lon, x.start_lat, x.end_lon, x.end_lat), axis=1 )
print('done')

In [ ]:
speed_df=speed_df[speed_df['distance']>0]
print('done')

In [ ]:
speed_df['direction']=speed_df.apply(lambda x: 'North' if x.end_lat>x.start_lat else 'South', axis=1)
print('done')

In [ ]:
speed_df['speed']=speed_df['distance']/speed_df['time_diff']
print('done')

In [ ]:
speed_df.head()

# Data Analysis

In [ ]:
df_mod=speed_df
df_mod_N=df_mod[df_mod['direction']=='North']
df_mod_S=df_mod[df_mod['direction']=='South']

df_mod_NO=df_mod_N[df_mod_N['month']==10]
df_mod_NO=df_mod_NO.resample(rule='5Min',on='start_time' ).mean()
df_mod_NO=df_mod_NO.reset_index()

df_mod_NN=df_mod_N[df_mod_N['month']==11]
df_mod_NN=df_mod_NN.resample(rule='5Min',on='start_time' ).mean()
df_mod_NN=df_mod_NN.reset_index()



df_mod_SO=df_mod_S[df_mod_S['month']==10]
df_mod_SO=df_mod_SO.resample(rule='5Min',on='start_time' ).mean()
df_mod_SO=df_mod_SO.reset_index()

df_mod_SN=df_mod_S[df_mod_S['month']==11]
df_mod_SN=df_mod_SN.resample(rule='5Min',on='start_time' ).mean()
df_mod_SN=df_mod_SN.reset_index()
print('done')

In [ ]:
fig,axs=plt.subplots(2,1,figsize=(12,7))
axs[0].plot(df_mod_NO['start_time'],df_mod_NO['speed'],label='North Direction')
axs[0].plot(df_mod_SO['start_time'],df_mod_SO['speed'],label='South Direction')

axs[1].plot(df_mod_NN['start_time'],df_mod_NN['speed'],label='North Direction')
axs[1].plot(df_mod_SN['start_time'],df_mod_SN['speed'],label='South Direction')


axs[0].xaxis.set_major_formatter(mdates.DateFormatter('%m-%d \n%H:%M %p'))
axs[0].xaxis.set_major_locator(mticker.MaxNLocator(15)) 

axs[1].xaxis.set_major_formatter(mdates.DateFormatter('%m-%d \n%H:%M %p'))
axs[1].xaxis.set_major_locator(mticker.MaxNLocator(15)) 

axs[0].set_ylabel('Speed (kmph)')
axs[0].legend(loc='upper right')


axs[1].set_ylabel('Speed (kmph)')
axs[1].legend(loc='upper right')

axs[0].set_xlabel('(a) For October, 2016',weight='bold')
axs[1].set_xlabel('(b) For November, 2016', weight='bold')

plt.tight_layout()
plt.savefig('5minplot.png',dpi=300)
plt.show()

In [ ]:
grp_mean=df_mod_N.groupby(by=[df_mod_N.start_time.map(lambda x : x.hour)]).speed.mean()
grp_std=df_mod_N.groupby(by=[df_mod_N.start_time.map(lambda x : x.hour)]).speed.std()


grp_mean_S=df_mod_S.groupby(by=[df_mod_S.start_time.map(lambda x : x.hour)]).speed.mean()
grp_std_S=df_mod_S.groupby(by=[df_mod_S.start_time.map(lambda x : x.hour)]).speed.std()
print('done')

In [ ]:
rng = pd.date_range('2017-04-03 00:00:00', periods=24, freq='H')
df = pd.DataFrame({'starttime': rng})  
cats = df.starttime.dt.strftime('%I %p').tolist()

fig,ax=plt.subplots(figsize=(10,7))

alpha=.4
ax.errorbar(range(24),grp_mean,yerr=grp_std,color='g',alpha=alpha,label='Standard Error(N)')
ax.plot(range(24),grp_mean,color='g',marker='d',markersize=10,label='North Direction(N)')

ax.errorbar(range(24),grp_mean_S,yerr=grp_std_S,color='r',alpha=alpha,label='Standard Error(S)')
ax.plot(range(24),grp_mean_S,color='r',marker='o',markersize=10,label='South Direction(S)')
ax.set_ylabel('Speed (kmph)')

tlabels=[]
for i in range(24):
    if i % 2==0:
        tlabels.append(cats[i])
    else:
        tlabels.append(' ')
plt.xticks(range(24),tlabels)

ax.legend()
plt.tight_layout()
plt.savefig('error_plot.png',dpi=300)
plt.show()

In [ ]:
df_mod_N.to_csv('model_data_N.csv')
df_mod_S.to_csv('model_data_S.csv')

# Fitting Models For North Direction

In [ ]:
df_mod_N=pd.read_csv('model_data_N.csv')
df_mod_N.head()

In [ ]:
df_mod_N=df_mod_N[['start_time','speed']]
df_mod_N['start_time']=pd.to_datetime(df_mod_N['start_time'])
df_mod_N.describe()

In [ ]:
df_mod_N5min=df_mod_N.resample(rule='5Min',on='start_time' ).mean()
df_mod_N5min=df_mod_N5min.reset_index()
df_mod_N5min.head()

In [ ]:
# Feature Creation
df_mod_N5min['hour']=df_mod_N5min.apply(lambda x: x.start_time.hour,axis=1)
df_mod_N5min['minute']=df_mod_N5min.apply(lambda x: x.start_time.minute,axis=1)
df_mod_N5min.head()

## LSTM Model

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler= MinMaxScaler(feature_range=(0, 1))
#scaler=StandardScaler()
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM
from keras import optimizers
from sklearn import model_selection
from keras.callbacks import EarlyStopping
from sklearn import neighbors
from math import sqrt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


In [ ]:
Model_List=[]
RMSEs=[]
for i in range(1,5*12+1):
    print('Working On Model : ', i)
    
    df_mod_N5min['lag1_speed']=df_mod_N5min['speed'].shift(i)
    df_mod_N5min['lag2_speed']=df_mod_N5min['speed'].shift(i+1)
    df_mod_N5min['lag3_speed']=df_mod_N5min['speed'].shift(i+2)
    df_mod_N5min['lag4_speed']=df_mod_N5min['speed'].shift(i+3)
    df_mod_N5min['lag5_speed']=df_mod_N5min['speed'].shift(i+4) 
    df_mod_N5min['lag6_speed']=df_mod_N5min['speed'].shift(i+5) 
    
    df_comb=df_mod_N5min[['start_time','hour','minute','lag1_speed','lag2_speed','lag3_speed','lag4_speed','lag5_speed','lag6_speed','speed']]
    df_comb=df_comb.dropna()
    df_comb_np=df_comb.values[:]
    test_size=int(.2*len(df_comb_np))

    times=df_comb_np[:,0]

    train_time=times[:-test_size]
    test_time=times[-test_size:]

    df_comb_np=df_comb_np[:,1:]

    df_comb_np_scaled=scaler.fit_transform(df_comb_np)

    X=df_comb_np_scaled[:,:-1]
    y=df_comb_np_scaled[:,-1]
    #x_train,x_test,y_train,y_test=model_selection.train_test_split(X, y, test_size=.2, random_state=50)
    train=df_comb_np_scaled[:-test_size]
    test=df_comb_np_scaled[-test_size:]

    x_train = train[:,:-1]
    x_test=test[:,:-1]
    y_train=train[:,-1]
    y_test=test[:,-1]


    x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
    x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
    print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

    epoc_size=25
    early_stopping_monitor = EarlyStopping(patience=1,monitor='val_loss',
                                  min_delta=0,
                                  verbose=0, mode='auto')

    model = Sequential()
    #model.add(LSTM(500,return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2])))

    opt=optimizers.adam(lr=.0001)
    #model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='tanh'))
    model.compile(loss='mse', optimizer=opt)
    # fit network
    #history = model.fit(x_train, y_train,batch_size=10, validation_data=(x_test, y_test), verbose=2, shuffle=True)

    history=model.fit(x_train, y_train,epochs=epoc_size,batch_size=6, validation_data=(x_test, y_test), verbose=1,callbacks=[early_stopping_monitor], shuffle=False)
    print('done')
    
    Model_List.append(model)

    # plot history
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()
    plt.show()


    import numpy as np
    test_X=x_train
    test_y=y_train

    # make a prediction
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    # invert scaling for forecast
    inv_yhat = np.concatenate((test_X[:, :],yhat), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,-1]
    train_yhat=inv_yhat

    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((test_X[:, :],test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,-1]
    # calculate RMSE
    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
    print('Train RMSE: %.3f' % rmse)


    test_X=x_test
    test_y=y_test

    # make a prediction
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    # invert scaling for forecast
    inv_yhat = np.concatenate((test_X[:, :],yhat), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,-1]
    test_yhat=inv_yhat

    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((test_X[:, :],test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,-1]
    # calculate RMSE
    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
    print('Test RMSE: %.3f' % rmse)
    RMSEs.append(rmse)
    fig=plt.figure(figsize=(15,7))
    plt.plot(range(len(inv_yhat[:100])),inv_yhat[:100],label='prediction')
    plt.plot(range(len(inv_y[:100])),inv_y[:100],label='actual')
    plt.legend()
    plt.show()

In [ ]:
fig=plt.figure(figsize=(10,6))
plt.plot(np.arange(1,len(RMSEs)+1)*5, np.array(RMSEs))
plt.xlabel('Forecast Horizon (Minute)')
plt.ylabel('RMSE (kmph)')
plt.show()

## Performance on Prediction data

In [ ]:
pred_data="Predictions_north.csv"
pred_df=pd.read_csv(pred_data)
len(pred_df)

df_comb_np_scaled.shape

pred_df['time_dt']=pd.to_datetime(pred_df['time'])

pred_df=pred_df.replace('x',10**6)
print(len(pred_df))
pred_df['speed']=pred_df['speed'].astype(float)

pred_df=pred_df.set_index('time_dt').resample('5Min').ffill().reset_index()
pred_df=pred_df.replace(10**6,np.nan)

fig=plt.subplots(figsize=(10,7))
plt.plot(pred_df['time_dt'],list(pred_df['speed']))
plt.show()

Predict_df=pred_df

Predict_df['hour']=Predict_df.apply(lambda x: x.time_dt.hour,axis=1)
Predict_df['minute']=Predict_df.apply(lambda x: x.time_dt.minute,axis=1)

Predict_df['lag1_speed']=Predict_df['speed']
Predict_df['lag2_speed']=Predict_df['speed'].shift(1)
Predict_df['lag3_speed']=Predict_df['speed'].shift(2)
Predict_df['lag4_speed']=Predict_df['speed'].shift(3)
Predict_df['lag5_speed']=Predict_df['speed'].shift(4)
Predict_df['lag6_speed']=Predict_df['speed'].shift(5)
Predict_df.head()

In [ ]:
test=Predict_df[['hour','minute','lag1_speed','lag2_speed','lag3_speed','lag4_speed','lag5_speed','lag6_speed','speed']].loc[72].values
test_scaled=scaler.transform(test.reshape(1,9))[:,:-1]
f_test_scaled_r=test_scaled.reshape(1,1,test_scaled.shape[1])

test=Predict_df[['hour','minute','lag1_speed','lag2_speed','lag3_speed','lag4_speed','lag5_speed','lag6_speed','speed']].loc[192].values
test_scaled=scaler.transform(test.reshape(1,9))[:,:-1]
s_test_scaled_r=test_scaled.reshape(1,1,test_scaled.shape[1])

In [ ]:
f_time=Predict_df.loc[72]['time_dt']
s_time=Predict_df.loc[192]['time_dt']

def find_index(x,f_time,s_time):
    time_diff1=(x.time_dt-f_time).total_seconds()/(60*5)
    time_diff2=(x.time_dt-s_time).total_seconds()/(60*5)
    if 1<=time_diff1 <61:
        index=time_diff1
        
    elif 0<time_diff2 <61:
       index=-time_diff2
    else:
        index=-99999
    return index

In [ ]:
Predict_df['model_index']=Predict_df.apply(lambda x: find_index(x,f_time,s_time), axis=1) 

In [ ]:
def make_prediction(model_index,time_stamp, f_test, s_test,Model_List):
    model_index=int(model_index)
    hour=time_stamp.hour
    minute=time_stamp.minute
    h_scaler=MinMaxScaler(feature_range=(0,1))
    h_scaler.fit_transform(np.array(Predict_df['hour']).reshape(-1,1))
    
    m_scaler=MinMaxScaler(feature_range=(0,1))
    m_scaler.fit_transform(np.array(Predict_df['minute']).reshape(-1,1))
    hour=h_scaler.transform(np.array(hour).reshape(-1,1))
    minute=m_scaler.transform(np.array(minute).reshape(-1,1))
    f_test_mod=f_test.reshape(1,8)
    f_test_mod[:,0]=hour
    f_test_mod[:,1]=minute
    f_test=f_test_mod.reshape(1,1,8)
    
    s_test_mod=s_test.reshape(1,8)
    s_test_mod[:,0]=hour
    s_test_mod[:,1]=minute
    s_test=s_test_mod.reshape(1,1,8)
    
    
 #   print(model_index)
    if model_index in range(1,61):
        pred_val=Model_List[model_index-1].predict(f_test)
        pred_val=new_scaler.inverse_transform(pred_val)[0][0]
    elif model_index in range(-61,0):
    #    print('mod',-model_index-1)
        pred_val=Model_List[-(model_index)-1].predict(s_test)
        pred_val=new_scaler.inverse_transform(pred_val)[0][0]
    else:
        pred_val=np.nan
    return pred_val

In [ ]:
        
Predict_df['prediction']=Predict_df.apply(lambda x: make_prediction(x.model_index,x.time_dt,f_test_scaled_r,s_test_scaled_r, Model_List), axis=1)


In [ ]:
fig,ax=plt.subplots(figsize=(20,7))
ax.plot(pred_df['time_dt'],list(pred_df['speed']),label='Given Speed Data')
ax.plot(Predict_df['time_dt'],list(Predict_df['prediction']),label='Prediction')
ax.set_xlim(pred_df['time_dt'].min(),pred_df['time_dt'].max())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M %p'))
plt.legend()
plt.show()


In [ ]:
Predict_df_N=Predict_df[['time','speed','prediction']]

Predict_df_N.to_csv('North_data_Prediction.csv',index=False)

In [ ]:
df=pd.merge(pred_df,Predict_df_N, how='left', on='time')

df=df.drop_duplicates(subset='time', keep="first")

df[['time','speed','prediction']].to_csv('Prediction_north_updated.csv')

# Fitting Models For South Direction

In [ ]:
df_mod_S=pd.read_csv('model_data_S.csv')
df_mod_S.head()

In [ ]:
df_mod_S=df_mod_S[['start_time','speed']]
df_mod_S['start_time']=pd.to_datetime(df_mod_S['start_time'])
df_mod_S.describe()

In [ ]:
df_mod_S5min=df_mod_S.resample(rule='5Min',on='start_time' ).mean()
df_mod_S5min=df_mod_S5min.reset_index()
df_mod_S5min.head()

In [ ]:
# Feature Creation
df_mod_S5min['hour']=df_mod_S5min.apply(lambda x: x.start_time.hour,axis=1)
df_mod_S5min['minute']=df_mod_S5min.apply(lambda x: x.start_time.minute,axis=1)
df_mod_S5min.head()

## LSTM

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler= MinMaxScaler(feature_range=(0, 1))
#scaler=StandardScaler()
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM
from keras import optimizers
from sklearn import model_selection
from keras.callbacks import EarlyStopping
from sklearn import neighbors
from math import sqrt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
Model_List=[]
RMSEs=[]
for i in range(1,5*12+1):
    print('Working On Model : ', i)
    
    df_mod_S5min['lag1_speed']=df_mod_S5min['speed'].shift(i)
    df_mod_S5min['lag2_speed']=df_mod_S5min['speed'].shift(i+1)
    df_mod_S5min['lag3_speed']=df_mod_S5min['speed'].shift(i+2)
    df_mod_S5min['lag4_speed']=df_mod_S5min['speed'].shift(i+3)
    df_mod_S5min['lag5_speed']=df_mod_S5min['speed'].shift(i+4) 
    df_mod_S5min['lag6_speed']=df_mod_S5min['speed'].shift(i+5) 
    
    df_comb=df_mod_S5min[['start_time','hour','minute','lag1_speed','lag2_speed','lag3_speed','lag4_speed','lag5_speed','lag6_speed','speed']]
    df_comb=df_comb.dropna()
    df_comb_np=df_comb.values[:]
    test_size=int(.2*len(df_comb_np))

    times=df_comb_np[:,0]

    train_time=times[:-test_size]
    test_time=times[-test_size:]

    df_comb_np=df_comb_np[:,1:]

    df_comb_np_scaled=scaler.fit_transform(df_comb_np)

    X=df_comb_np_scaled[:,:-1]
    y=df_comb_np_scaled[:,-1]
    #x_train,x_test,y_train,y_test=model_selection.train_test_split(X, y, test_size=.2, random_state=50)
    train=df_comb_np_scaled[:-test_size]
    test=df_comb_np_scaled[-test_size:]

    x_train = train[:,:-1]
    x_test=test[:,:-1]
    y_train=train[:,-1]
    y_test=test[:,-1]


    x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
    x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
    print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

    epoc_size=25
    early_stopping_monitor = EarlyStopping(patience=1,monitor='val_loss',
                                  min_delta=0,
                                  verbose=0, mode='auto')

    model = Sequential()
    #model.add(LSTM(500,return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2])))

    opt=optimizers.adam(lr=.0001)
    #model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='tanh'))
    model.compile(loss='mse', optimizer=opt)
    # fit network
    #history = model.fit(x_train, y_train,batch_size=10, validation_data=(x_test, y_test), verbose=2, shuffle=True)

    history=model.fit(x_train, y_train,epochs=epoc_size,batch_size=6, validation_data=(x_test, y_test), verbose=1,callbacks=[early_stopping_monitor], shuffle=False)
    print('done')
    
    Model_List.append(model)

    # plot history
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()
    plt.show()


    import numpy as np
    test_X=x_train
    test_y=y_train

    # make a prediction
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    # invert scaling for forecast
    inv_yhat = np.concatenate((test_X[:, :],yhat), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,-1]
    train_yhat=inv_yhat

    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((test_X[:, :],test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,-1]
    # calculate RMSE
    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
    print('Train RMSE: %.3f' % rmse)


    test_X=x_test
    test_y=y_test

    # make a prediction
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    # invert scaling for forecast
    inv_yhat = np.concatenate((test_X[:, :],yhat), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,-1]
    test_yhat=inv_yhat

    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((test_X[:, :],test_y), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,-1]
    # calculate RMSE
    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
    print('Test RMSE: %.3f' % rmse)
    RMSEs.append(rmse)
    fig=plt.figure(figsize=(15,7))
    plt.plot(range(len(inv_yhat[:100])),inv_yhat[:100],label='prediction')
    plt.plot(range(len(inv_y[:100])),inv_y[:100],label='actual')
    plt.legend()
    plt.show()

In [ ]:
fig=plt.figure(figsize=(10,6))
plt.plot(np.arange(1,len(RMSEs)+1)*5, np.array(RMSEs))
plt.xlabel('Forecast Horizon (Minute)')
plt.ylabel('RMSE (kmph)')
plt.show()

## Performance on Prediction Data

In [ ]:
pred_data="Predictions_south.csv"
pred_df=pd.read_csv(pred_data)
len(pred_df)

df_comb_np_scaled.shape

pred_df['time_dt']=pd.to_datetime(pred_df['time'])

pred_df=pred_df.replace('x',10**6)
print(len(pred_df))
pred_df['speed']=pred_df['speed'].astype(float)

pred_df=pred_df.set_index('time_dt').resample('5Min').ffill().reset_index()
pred_df=pred_df.replace(10**6,np.nan)

fig=plt.subplots(figsize=(10,7))
plt.plot(pred_df['time_dt'],list(pred_df['speed']))
plt.show()

Predict_df=pred_df

Predict_df['hour']=Predict_df.apply(lambda x: x.time_dt.hour,axis=1)
Predict_df['minute']=Predict_df.apply(lambda x: x.time_dt.minute,axis=1)

Predict_df['lag1_speed']=Predict_df['speed']
Predict_df['lag2_speed']=Predict_df['speed'].shift(1)
Predict_df['lag3_speed']=Predict_df['speed'].shift(2)
Predict_df['lag4_speed']=Predict_df['speed'].shift(3)
Predict_df['lag5_speed']=Predict_df['speed'].shift(4)
Predict_df['lag6_speed']=Predict_df['speed'].shift(5)
Predict_df.head()

In [ ]:
test=Predict_df[['hour','minute','lag1_speed','lag2_speed','lag3_speed','lag4_speed','lag5_speed','lag6_speed','speed']].loc[72].values
test_scaled=scaler.transform(test.reshape(1,9))[:,:-1]
f_test_scaled_r=test_scaled.reshape(1,1,test_scaled.shape[1])

test=Predict_df[['hour','minute','lag1_speed','lag2_speed','lag3_speed','lag4_speed','lag5_speed','lag6_speed','speed']].loc[192].values
test_scaled=scaler.transform(test.reshape(1,9))[:,:-1]
s_test_scaled_r=test_scaled.reshape(1,1,test_scaled.shape[1])

In [ ]:
f_time=Predict_df.loc[72]['time_dt']
s_time=Predict_df.loc[192]['time_dt']

def find_index(x,f_time,s_time):
    time_diff1=(x.time_dt-f_time).total_seconds()/(60*5)
    time_diff2=(x.time_dt-s_time).total_seconds()/(60*5)
    if 1<=time_diff1 <61:
        index=time_diff1
        
    elif 0<time_diff2 <61:
       index=-time_diff2
    else:
        index=-99999
    return index

In [ ]:
Predict_df['model_index']=Predict_df.apply(lambda x: find_index(x,f_time,s_time), axis=1) 

In [ ]:
def make_prediction(model_index,time_stamp, f_test, s_test,Model_List):
    model_index=int(model_index)
    hour=time_stamp.hour
    minute=time_stamp.minute
    h_scaler=MinMaxScaler(feature_range=(0,1))
    h_scaler.fit_transform(np.array(Predict_df['hour']).reshape(-1,1))
    
    m_scaler=MinMaxScaler(feature_range=(0,1))
    m_scaler.fit_transform(np.array(Predict_df['minute']).reshape(-1,1))
    hour=h_scaler.transform(np.array(hour).reshape(-1,1))
    minute=m_scaler.transform(np.array(minute).reshape(-1,1))
    f_test_mod=f_test.reshape(1,8)
    f_test_mod[:,0]=hour
    f_test_mod[:,1]=minute
    f_test=f_test_mod.reshape(1,1,8)
    
    s_test_mod=s_test.reshape(1,8)
    s_test_mod[:,0]=hour
    s_test_mod[:,1]=minute
    s_test=s_test_mod.reshape(1,1,8)
    
    
 #   print(model_index)
    if model_index in range(1,61):
        pred_val=Model_List[model_index-1].predict(f_test)
        pred_val=new_scaler.inverse_transform(pred_val)[0][0]
    elif model_index in range(-61,0):
    #    print('mod',-model_index-1)
        pred_val=Model_List[-(model_index)-1].predict(s_test)
        pred_val=new_scaler.inverse_transform(pred_val)[0][0]
    else:
        pred_val=np.nan
    return pred_val

In [ ]:
Predict_df['prediction']=Predict_df.apply(lambda x: make_prediction(x.model_index,x.time_dt,f_test_scaled_r,s_test_scaled_r, Model_List), axis=1)

In [ ]:
fig,ax=plt.subplots(figsize=(20,7))
ax.plot(pred_df['time_dt'],list(pred_df['speed']),label='Given Speed Data')
ax.plot(Predict_df['time_dt'],list(Predict_df['prediction']),label='Prediction')
ax.set_xlim(pred_df['time_dt'].min(),pred_df['time_dt'].max())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M %p'))
plt.legend()
plt.show()

In [ ]:
Predict_df_S=Predict_df[['time','speed','prediction']]

Predict_df_S.to_csv('South_data_Prediction.csv',index=False)

In [ ]:
df=pd.merge(pred_df,Predict_df_S, how='left', on='time')

df=df.drop_duplicates(subset='time', keep="first")

df[['time','speed','prediction']].to_csv('Prediction_south_updated.csv')

In [ ]:
### Cheers